In [1]:
import tensorflow as tf
tf.executing_eagerly()

True

In [2]:
#Used in Tensorflow Model
import numpy as np
import tflearn
import random

#Used to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
warnings.filterwarnings("ignore")
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
2.6.0-dev20210528


In [3]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [5]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
51 documents
8 classes ['goodbye', 'greeting', 'jokes', 'kosong', 'moodbaik', 'moodburuk', 'perkenalan', 'thanks']
86 unique stemmed words ['!', ',', 'ad', 'aj', 'akhirny', 'aku', 'antusia', 'ap', 'apakah', 'bahag', 'banget', 'banyak', 'bersemang', 'bersyuk', 'bis', 'boleh', 'bro', 'capek', 'cer', 'dadah', 'deng', 'dengerin', 'dir', 'dulu', 'feel', 'ga', 'galau', 'good', 'hai', 'hal', 'halo', 'har', 'hello', 'hey', 'hi', 'in', 'ingin', 'itu', 'iâ€™m', 'jad', 'jok', 'jug', 'jump', 'kab', 'kamu', 'kasih', 'kegy', 'ken', 'kesel', 'kurang', 'lag', 'lelucon', 'lib', 'lucu', 'makasih', 'mau', 'membantu', 'menyelesaik', 'nam', 'nang', 'nyerah', 'obrol', 'orang', 'overthink', 'percay', 'perkenalk', 'rasany', 'sampa', 'sang', 'selesa', 'semang', 'senang', 'siap', 'stress', 'sudah', 'terim', 'terimakasih', 'terserah', 'thank', 'tid', 'tugasku', 'up', 'what', 'ya', 'yang', 'ye']


In [6]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [7]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [8]:
# %tensorflow_version 1.x
tf.compat.v1.reset_default_graph()

In [9]:
#model versi tflearn
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax', name='my_output')
net = tflearn.regression(net, learning_rate = 2,
        optimizer = 'sgd',
        loss = 'mean_square')
print("Training....")

try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
    print("Training the Model.......")
    model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)
    print("Saving the Model.......")
    with net.graph.as_default():
        del tf.compat.v1.get_collection_ref(tf.compat.v1.GraphKeys.TRAIN_OPS)[:]
    model.save('model.tflearn')
#     model.save('chatbot_model.h5')

Training Step: 3499  | total loss: 0.00106 | time: 0.043s
| SGD | epoch: 500 | loss: 0.00106 - acc: 1.0000 -- iter: 48/51
Training Step: 3500  | total loss: 0.00102 | time: 0.048s
| SGD | epoch: 500 | loss: 0.00102 - acc: 1.0000 -- iter: 51/51
--
Saving the Model.......
INFO:tensorflow:C:\Users\Noeroel Nabilah\Desktop\Folder Desktop\Bangkit\Final Project\ML-Chatbot\chatbot_model\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [11]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from C:\Users\Noeroel Nabilah\Desktop\Folder Desktop\Bangkit\Final Project\ML-Chatbot\chatbot_model\model.tflearn


In [12]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [13]:
from time import sleep
print("Hai, hari yang indah")
sleep(0.5)
print("Perkenalkan, saya Zassy. Chatbot pribadi anda")
sleep(0.5)
print("Ada yang bisa saya bantu?")
while True:
    input_data = input("You : ")
    if input_data.lower()== "quit" :
        break
    answer = response(input_data)
    answer

Hai, hari yang indah
Perkenalkan, saya Zassy. Chatbot pribadi anda
Ada yang bisa saya bantu?
You : hai
Hai, senang sekali bisa chat dengan kamu. Ada yang bisa saya bantu?
You : senang
Kenapa kalo zombie nyerangnya bareng? Kalo sendiri Namanya zomblo
You : kok?
Sepertinya kamu belum mengetik, bisa jelaskan pesannya kesaya?
You : aku lagi senang
wah, kamu terlihat cerah dan senang hari ini. Apa yang membuat kamu senang?
You : jokes
kalo semua hal harus dipikirkan masak-masak, gimana coba nasib orang-orang yang gabisa masak
You : quit
